# Create a .env file and store this

OPENAI_API_KEY="your-key-here"

## String knowledge source

In [1]:
from crewai.knowledge.source.string_knowledge_source import StringKnowledgeSource

# Define the knowledge
policy_text = """Our return policy allows customers to return any product within 30 days of purchase.
                 Refunds will be issued only if the item is unused and in original packaging.
                 Customers must provide proof of purchase when requesting a return."""

# Create a StringKnowledgeSource object
return_policy_knowledge = StringKnowledgeSource(content=policy_text)


In [2]:
from crewai import LLM
from dotenv import load_dotenv
load_dotenv()

llm = LLM(model="gpt-4o")


In [3]:
from crewai import Agent

returns_agent = Agent(
    role="Product Returns Assistant",
    goal="Answer customer questions about return policy accurately.",
    backstory="You work in customer service and specialize in returns, refunds, and policies.",
    allow_delegation=False,
    verbose=True,
    llm=llm
)


In [4]:
from crewai import Task

returns_task = Task(
    description="Answer the following customer question about returns: {question}",
    expected_output="A concise and accurate answer.",
    agent=returns_agent
)


In [8]:
from crewai import Crew, Process

crew = Crew(
    agents=[returns_agent],
    tasks=[returns_task],
    process=Process.sequential,
    knowledge_sources=[return_policy_knowledge],  # This is key
    verbose=False
)


In [ ]:
result = crew.kickoff(inputs={
    "question": "Can I get a retrun an item after 45 days?"
})

from pprint import pprint
pprint(result.raw)

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Product Returns Assistant                                                                               │
│                                                                                                                 │
│  Task: Answer the following customer question about returns: Can I get a retrun an item after 45 days?          │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Product Returns Assistant                                                                               │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  Our return policy allows customers to return any product within 30 days of purchase. Since it has been 45      │
│  days since your purchase, unfortunately, we cannot accept the return. Refunds are only issued for returns      │
│  made within 30 days, provided the item is unused and in its original packaging, along with proof of purchase.  │
│  If you have further questions or need assistance, please let us know.                                          │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

('Our return policy allows customers to return any product within 30 days of '
 'purchase. Since it has been 45 days since your purchase, unfortunately, we '
 'cannot accept the return. Refunds are only issued for returns made within 30 '
 'days, provided the item is unused and in its original packaging, along with '
 'proof of purchase. If you have further questions or need assistance, please '
 'let us know.')


## Text Knowledge Source

In [10]:
from crewai.knowledge.source.text_file_knowledge_source import TextFileKnowledgeSource

text_source = TextFileKnowledgeSource(
    file_paths=["hr_policy.txt"]
)

In [13]:
from crewai import Agent, Task, Crew, Process, LLM

llm = LLM(model="gpt-4o")

hr_agent = Agent(
    role="HR Policy Assistant",
    goal="Answer employee questions about HR policies.",
    backstory="You're a reliable HR knowledge assistant.",
    knowledge_sources=[text_source],
    llm=llm
)

task = Task(
    description="Answer the following HR policy question: {question}",
    expected_output="A concise and accurate answer based on the HR policy.",
    agent=hr_agent
)


In [14]:
crew = Crew(
    agents=[hr_agent],
    tasks=[task],
    process=Process.sequential,
    verbose=False
)

result = crew.kickoff(inputs={
    "question": "What's sick leave policy for new employees?"
})
pprint(result.raw)


('New employees are eligible for sick leave after completing their first 30 '
 'days of employment. Sick leave must be reported on the same day before 10 '
 'AM. If the sick leave exceeds 3 consecutive days, the employee must provide '
 'supporting documentation.')


## PDF source

In [15]:
from crewai.knowledge.source.pdf_knowledge_source import PDFKnowledgeSource

pdf_source = PDFKnowledgeSource(
    file_paths=["meeting_notes.pdf"]
)

In [16]:
meeting_summarizer = Agent(
    role="Meeting Note Summarizer",
    goal="Provide concise summaries of weekly meetings.",
    backstory="You help the team stay updated on discussions.",
    knowledge_sources=[pdf_source],
    llm=llm
)

task = Task(
    description="Summarize the key action items from last week's meeting.",
    expected_output="A bullet-point list of action items.",
    agent=meeting_summarizer
)


In [17]:
crew = Crew(
    agents=[meeting_summarizer],
    tasks=[task],
    process=Process.sequential,
    verbose=True
)

result = crew.kickoff()
pprint(result.raw)


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 91103d2d-2a70-4323-be41-b38d9913cc88                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

d:\Learn\AI-Agents-Dev\DDDS-Agent-Course\.venv\Lib\site-packages\rich\live.py:256: UserWarning: install 
"ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

╭──────────────────────────────────────────── 📚 Retrieved Knowledge ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Additional Information: Weekly Strategy Sync - April 12, 2025                                                  │
│  Participants:                                                                                                  │
│  - CTO, Head of Product, Data Science Lead, Engineering Manager                                                 │
│  Discussion Summary:                                                                                            │
│  - Launch of new AI recommendation engine delayed by 1 week due to testing issues.                              │
│  - Product team to finalize Q2 roadmap by April 20.                                                             │
│  - DS team to evaluate CrewAI as a potential integration partner.                                               │
│  - Engineering to move remaining microservices to Kubernetes by end of May.                                     │
│  Action Items:                                                                                                  │
│  1. Product: Finalize Q2 roadmap (due April 20)                                                                 │
│  2. D...                                                                                                        │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Meeting Note Summarizer                                                                                 │
│                                                                                                                 │
│  Task: Summarize the key action items from last week's meeting.                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Meeting Note Summarizer                                                                                 │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  - Product: Finalize Q2 roadmap (due April 20)                                                                  │
│  - DS: Conduct feasibility study on CrewAI (due April 18)                                                       │
│  - Eng: Migrate ML serving infra to Kubernetes (due May 31)                                                     │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 73721b7d-2fc6-4670-a99a-c3b6c933d6b6                                                                     │
│  Agent: Meeting Note Summarizer                                                                                 │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 91103d2d-2a70-4323-be41-b38d9913cc88                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: - Product: Finalize Q2 roadmap (due April 20)                                                    │
│  - DS: Conduct feasibility study on CrewAI (due April 18)                                                       │
│  - Eng: Migrate ML serving infra to Kubernetes (due May 31)                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

('- Product: Finalize Q2 roadmap (due April 20)\n'
 '- DS: Conduct feasibility study on CrewAI (due April 18)\n'
 '- Eng: Migrate ML serving infra to Kubernetes (due May 31)')


## CSV source

In [18]:
from crewai.knowledge.source.csv_knowledge_source import CSVKnowledgeSource

csv_source = CSVKnowledgeSource(
    file_paths=["feedback.csv"]
)

In [19]:
feedback_analyst = Agent(
    role="User Feedback Analyst",
    goal="Identify common themes in user feedback.",
    backstory="You specialize in converting raw feedback into insights.",
    knowledge_sources=[csv_source],
    llm=llm
)

task = Task(
    description="What are the three most common complaints users had last month?",
    expected_output="A short list of recurring issues.",
    agent=feedback_analyst
)

In [21]:
crew = Crew(
    agents=[feedback_analyst],
    tasks=[task],
    process=Process.sequential,
    verbose=False
)

result = crew.kickoff()
pprint(result.raw)


('The three most common complaints users had last month are: 1) The UI feels '
 'slow on mobile. 2) Notifications are too frequent and not customizable. 3) '
 'It’s hard to find the export option in reports.')


## JSON source

In [22]:
from crewai.knowledge.source.json_knowledge_source import JSONKnowledgeSource

json_source = JSONKnowledgeSource(
    file_paths=["company_info.json"]
)

In [23]:
company_expert = Agent(
    role="Company Info Specialist",
    goal="Answer questions about company structure and data.",
    backstory="You are an internal data assistant for org-level queries.",
    # knowledge_sources=[json_source],
    llm=llm
)

task = Task(
    description="How many teams are working on the product and what are their names?",
    expected_output="A list of team names and their sizes.",
    agent=company_expert
)


In [24]:
crew = Crew(
    agents=[company_expert],
    tasks=[task],
    process=Process.sequential,
    verbose=False,
    knowledge_sources=[json_source]
)

result = crew.kickoff()
print(result)


The teams working on the product for TechNova Inc. are as follows:
1. UX Team - 5 members
2. Analytics Team - 3 members


## Custom embedding model

In [1]:
ollama_embedder = {
    "provider": "ollama",
    "config": {
        "model": "nomic-embed-text",  # Must match or be compatible with Ollama's supported embedding models
        "api_url": "http://localhost:11434"
    }
}

In [2]:
from crewai.knowledge.source.string_knowledge_source import StringKnowledgeSource

# Internal onboarding FAQ
faq_content = """
- You can access your email via portal.company.com using your employee credentials.
- The standard work hours are from 9am to 6pm, Monday to Friday.
- All reimbursement requests must be submitted by the 5th of the following month.
- For any IT-related issues, contact support@company.com.
"""

# Create a string knowledge source
faq_knowledge = StringKnowledgeSource(content=faq_content, embedder=ollama_embedder)


In [3]:
from crewai import Agent

hr_faq_agent = Agent(
    role="HR Assistant",
    goal="Answer onboarding-related questions for new hires.",
    backstory="You are a helpful assistant who knows everything about internal policies and onboarding processes.",
    allow_delegation=False,
    verbose=True,
    embedder=ollama_embedder
)


In [4]:
from crewai import Task

task = Task(
    description="Answer this onboarding question: {question}",
    expected_output="A short, accurate answer based on internal HR documentation.",
    agent=hr_faq_agent,
    embedder=ollama_embedder
)


In [ ]:
from crewai import Crew, Process

crew = Crew(
    agents=[hr_faq_agent],
    tasks=[task],
    knowledge_sources=[faq_knowledge],
    embedder=ollama_embedder,
    process=Process.sequential,
    verbose=True
)

result = crew.kickoff(inputs={
    "question": "What are the working hours and how do I get reimbursed?"
})

from pprint import pprint
pprint(result.raw)


# Custom knowledge source

In [25]:
from crewai.knowledge.source.base_knowledge_source import BaseKnowledgeSource
from typing import Dict, Any
from pydantic import Field
import requests

class WeatherKnowledgeSource(BaseKnowledgeSource):
    """Knowledge source that fetches weather data from an external API."""

    city: str = Field(description="City for which weather should be fetched")

    def load_content(self) -> Dict[Any, str]:
        try:
            print(f"Fetching weather for {self.city}...")

            # Open-Meteo API (no key needed for basic data)
            endpoint = "https://api.open-meteo.com/v1/forecast"
            params = {
                "latitude": 37.77,  # San Francisco by default
                "longitude": -122.42,
                "current_weather": True
            }

            response = requests.get(endpoint, params=params)
            response.raise_for_status()

            weather_data = response.json().get("current_weather", {})
            formatted = self.validate_content(weather_data)
            return {self.city: formatted}

        except Exception as e:
            raise ValueError(f"Failed to fetch weather data: {str(e)}")

    def validate_content(self, data: dict) -> str:
        if not data:
            return "No weather data available."

        return (
            f"Current weather in {self.city}:\n"
            f"- Temperature: {data.get('temperature')}°C\n"
            f"- Wind Speed: {data.get('windspeed')} km/h\n"
            f"- Weather Code: {data.get('weathercode')}\n"
            f"- Time: {data.get('time')}"
        )

    def add(self) -> None:
        """Process and chunk the content."""
        content = self.load_content()
        for _, text in content.items():
            chunks = self._chunk_text(text)
            self.chunks.extend(chunks)
        self._save_documents()


In [34]:
from crewai import Agent, LLM

weather_knowledge = WeatherKnowledgeSource(city="San Francisco")

weather_agent = Agent(
    role="Weather Reporter",
    goal="Answer questions about the current weather forecast.",
    backstory="You are a friendly meteorologist who provides real-time weather updates.",
    knowledge_sources=[weather_knowledge],
    llm=LLM(model="gpt-4o", temperature=0.0),
    verbose=False
)


In [37]:
from crewai import Task, Crew, Process

task = Task(
    description="What is the current temperature and wind speed in San Francisco?",
    expected_output="A concise weather summary for San Francisco.",
    agent=weather_agent
)

crew = Crew(
    agents=[weather_agent],
    tasks=[task],
    process=Process.sequential,
    verbose=False
)


In [38]:
result = crew.kickoff()
print(result)


Fetching weather for San Francisco...
The current temperature in San Francisco is 15.9°C with a wind speed of 14.4 km/h. The weather code is 3, indicating partly cloudy conditions. The time of this report is 17:15 on July 26, 2025.
